<a href="https://colab.research.google.com/github/danielegenta/Progetto-MLDL/blob/lwf/main_lwf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
  Following the iCaRL paper specifications,
  LwF is implemented simirality to iCaRL itself.
  The differences are:
  - No exemplars management
  - For classification, it is used the network output values themselves
  (ref. iCaRL paper section 4.1)
"""

'\n  Following the iCaRL paper specifications,\n  LwF is implemented simirality to iCaRL itself.\n  The differences are:\n  - No exemplars management\n  - For classification, it is used the network output values themselves\n  (ref. iCaRL paper section 4.1)\n'

In [2]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms

from PIL import Image
from tqdm import tqdm
import random


import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import time

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
DATA_DIR = 'DATA' # here the dataset will be downloaded

NUM_CLASSES = 10 

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 2           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 70       # Total number of training epochs (iterations over dataset)
MILESTONES = [49, 63] # when the LR decreases, according to icarl
GAMMA = 0.1           # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

RANDOM_SEED = 30 # implement this! It will be easier to do 3 different trials

In [4]:
# Clone github repository with dataset handler
!rm -r Cifar100/
!rm -r $DATA_DIR
!mkdir "DATA"
if not os.path.isdir('./Cifar100'):
  !git clone -b lwf https://github.com/danielegenta/Progetto-MLDL.git
  !mv 'Progetto-MLDL' 'Cifar100'
  !rm -r Cifar100/Theoretical-Sources
  !rm -rf Cifar100/ProjectMLDL.ipynb

Cloning into 'Progetto-MLDL'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 311 (delta 26), reused 30 (delta 14), pack-reused 269
Receiving objects: 100% (311/311), 3.75 MiB | 3.33 MiB/s, done.
Resolving deltas: 100% (162/162), done.


In [5]:
# Download dataset from the official source and save it into DATA/cifar-100-pyhton

if not os.path.isdir('./{}'.format("$DATA_DIR/cifar-100-python")):
    !wget https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
    !tar -xf 'cifar-100-python.tar.gz'  
    !mkdir $DATA_DIR
    !mv 'cifar-100-python' "$DATA_DIR/cifar-100-python"
    !rm -rf 'cifar-100-python.tar.gz'

--2020-05-24 22:01:13--  https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169001437 (161M) [application/x-gzip]
Saving to: ‘cifar-100-python.tar.gz’

cifar-100-python.ta 100%[===================>] 161.17M  16.5MB/s    in 11s     

2020-05-24 22:01:25 (14.6 MB/s) - ‘cifar-100-python.tar.gz’ saved [169001437/169001437]

mkdir: cannot create directory ‘DATA’: File exists


In [0]:
# i could use cifar100 mean and std

# Define transforms for training phase
train_transform = transforms.Compose([
                                      transforms.Pad(4),         # Add padding
                                      transforms.RandomCrop(32), # Crops a random squares of the image  
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
                                      ])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                 
])

In [7]:
from Cifar100.Dataset.cifar100 import CIFAR100


# Import dataset
train_dataset = CIFAR100(DATA_DIR, split='train', transform=train_transform)
test_dataset = CIFAR100(DATA_DIR, split='test', transform=eval_transform)

# check if datasets have been correctly loaded
print(len(train_dataset))
print(len(test_dataset))

50000
10000


In [0]:
from Cifar100.reverse_index import ReverseIndex

def build_test_splits(dataset, reverse_index):
    splits = dict()
    groups = list(reverse_index.getGroups())
    for g in groups:
        labels_of_groups = reverse_index.getLabelsOfGroup(g)
        indices = list(dataset.df[dataset.df['labels'].isin(labels_of_groups)].index)
        splits[g] = indices
    return splits

In [0]:
# performing the train/val split
train_splits = train_dataset.split_in_train_val_groups(ratio=0.99, seed=30)
outputs_labels_mapping = ReverseIndex(train_dataset, train_splits)

# performing the test split (coherent with train/val)
test_splits = build_test_splits(test_dataset, outputs_labels_mapping)

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_subsets = []
val_subsets = []
test_subsets = []

for v in train_splits.values():
    train_subs = Subset(train_dataset, v['train'])
    val_subs = Subset(train_dataset, v['val'])
    train_subsets.append(train_subs)
    val_subsets.append(val_subs)

for i in range(0,10):
    v=test_splits[i]
    test_subs = Subset(test_dataset, v)
    test_subsets.append(test_subs)

**LWF implementation**

In [11]:
# default params

from Cifar100.lwf_model import LWF

feature_size = 64
n_classes = 1
lwf = LWF(feature_size, n_classes, BATCH_SIZE, WEIGHT_DECAY, LR, GAMMA, NUM_EPOCHS, DEVICE,MILESTONES)
lwf.to(DEVICE)

LWF(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, 

In [0]:
def incrementalTraining(net, train_subsets, val_subsets, test_subsets,eval_transform):
    for train_subset, val_subset, test_subset in zip(train_subsets, val_subsets, test_subsets):

      train_dataloader = DataLoader(train_subset, batch_size=BATCH_SIZE,shuffle=True, num_workers=4)
      val_dataloader = DataLoader(val_subset, batch_size=BATCH_SIZE,shuffle=False, num_workers=4)
      test_dataloader = DataLoader(test_subset, batch_size=BATCH_SIZE,shuffle=False, num_workers=4)
      
      new_classes_examined = list(train_dataset.df.loc[train_subset.indices, 'labels'].value_counts().index)
      
      net.train()

      # update representation
      net.update_representation(train_subset, new_classes_examined)

      net.n_known = net.n_classes

      # evaluation on the train set

      net.eval()

      print ("the model knows %d classes:\n " % net.n_known)
  
      #Evaluating on train set
      total = 0.0
      correct = 0.0

      for images, labels, indices in train_dataloader:
        images = images.to(self.DEVICE)
        preds = net.classify(images)
        total += labels.size(0)
        correct += (preds.data.cpu() == labels).sum()

      # Train Accuracy
      print ('Train Accuracy : %.2f\n' % (100.0 * correct / total))

      

In [13]:
incrementalTraining(lwf, train_subsets, val_subsets, test_subsets,eval_transform)

RuntimeError: ignored